In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import time

In [2]:
# Load the IMDB dataset
max_features = 5000
maxlen = 100
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)


In [3]:
# Preprocess the data
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [4]:
# Convert sequences back into text
reverse_word_index = dict([(value, key) for (key, value) in imdb.get_word_index().items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

X_train_text = [decode_review(x) for x in X_train]
X_test_text = [decode_review(x) for x in X_test]

# Create and fit tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train_text)

# Preprocess the data
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=maxlen)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=maxlen)

In [5]:
# Create and fit tokenizer
#tokenizer = Tokenizer(num_words=max_features)
#tokenizer.fit_on_texts(X_train)


In [6]:
# Build the model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


C:\Users\Kaustubh\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [8]:
# Train the model
batch_size = 32
epochs = 5
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

In [9]:
start_time = time.time()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)
training_time = time.time() - start_time


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 35s 40ms/step - accuracy: 0.7138 - loss: 0.5443 - val_accuracy: 0.8246 - val_loss: 0.3967
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 39ms/step - accuracy: 0.8543 - loss: 0.3490 - val_accuracy: 0.8130 - val_loss: 0.4154
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.8772 - loss: 0.2979 - val_accuracy: 0.8378 - val_loss: 0.3675
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 39ms/step - accuracy: 0.8979 - loss: 0.2504 - val_accuracy: 0.8335 - val_loss: 0.3924
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.9143 - loss: 0.2177 - val_accuracy: 0.8395 - val_loss: 0.3851


In [10]:
# Evaluate the model
start_time = time.time()

score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
prediction_time = time.time() - start_time

print("Test accuracy:", acc)

782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8385 - loss: 0.3942
Test accuracy: 0.8395199775695801


In [11]:
#print("Test accuracy:", acc)
#print("Training time:", training_time, "seconds")
#print("Prediction time:", prediction_time, "seconds")

In [12]:
# Function to preprocess user input
def preprocess_input(user_input):
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    sequences = tokenizer.texts_to_sequences(user_input)
    return pad_sequences(sequences, maxlen=maxlen)
   
#from tensorflow.keras.preprocessing.text import Tokenizer
    #tokenizer = Tokenizer(num_words=max_features)
    #tokenizer.fit_on_texts(user_input)
    #sequences = tokenizer.texts_to_sequences(user_input)
    #return pad_sequences(sequences, maxlen=maxlen)

In [13]:
# Function to predict sentiment
def predict_sentiment(review):
    review = preprocess_input([review])
    prediction = model.predict(review)[0][0]
    if prediction >= 0.5:
        return "Positive"
    else:
        return "Negative"

In [14]:
# User Interface
print("Welcome to Movie Review Sentiment Analyzer!")

Welcome to Movie Review Sentiment Analyzer!


In [15]:
while True:
    user_review = input("Enter your movie review (or type 'quit' to exit): ")
    if user_review.lower() == 'quit':
        print("Thank you for using our service!")
        break
    else:
        sentiment = predict_sentiment(user_review)
        print("Predicted sentiment:", sentiment)

Enter your movie review (or type 'quit' to exit):  The screenplay, cuts and sound effects deserves a standing ovation. There are illogical moments but that don't take away you, because the Director keeps you invested in the 'state of disbelief' and that is the success of this movie. There is hardly any sugarcoating to the criminal angle of Rocky, and it is more of a man with a promise to fulfill. Kudos to the entire cast and crew for a honest 'masala movie'!!


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
Predicted sentiment: Negative


Enter your movie review (or type 'quit' to exit):  Mindblownnnnn!!! What I saw on screen was the conviction and hardwork of the entire team that has worked with so much passion!! 👏🏻👏🏻👏🏻 Congrats on the well deserved humongous success.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted sentiment: Positive


Enter your movie review (or type 'quit' to exit):  It is one of the best movies and a history for indian cinemas.  Ot has grate actions and emotions and blah blah blah. It is a gold for indian cinemas.


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted sentiment: Negative


Enter your movie review (or type 'quit' to exit):   Great direction music and high tone acting . Just watch and experience movie as it is. Action drama crime at high level 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted sentiment: Positive


KeyboardInterrupt: Interrupted by user

Enter your movie review (or type 'quit' to exit):  quit
